In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import r2_score

import os
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Sequential,load_model,save_model
from tensorflow.keras.layers import Dense,Conv2D,Flatten,MaxPooling2D
from keras.layers import BatchNormalization
from keras.optimizers import Adam

Age Estimation

In [ ]:
# image_dir = Path('/content/drive/MyDrive/AI/.kaggle/20-50') #tell python in which directory the training images are.
image_dir = Path('/content/drive/MyDrive/AI/.kaggle/20-50')

In [ ]:
filepaths = pd.Series(list(image_dir.glob(r'**/*.jpg')), name='Filepath').astype(str)
ages = pd.Series(filepaths.apply(lambda x: os.path.split(os.path.split(x)[0])[1]), name='Age').astype(np.int32)

images = pd.concat([filepaths, ages], axis=1).sample(frac=1.0, random_state=1).reset_index(drop=True)


In [ ]:
image_df = images.sample(20000, random_state=1).reset_index(drop=True)

train_df, test_df = train_test_split(image_df, train_size=0.7, shuffle=True, random_state=1)

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath', # kepek
    y_col='Age', # becsülendő életkorok
    target_size=(120, 120), # egy kép mérete
    color_mode='rgb', # 3 csatornás képek, RGB
    class_mode='raw', # mivel a célváltozónk int típusú és nem object
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Age',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Age',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=False
)

In [ ]:
inputs = tf.keras.Input(shape=(120, 120, 3))
x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='linear')(x)

age_model = tf.keras.Model(inputs=inputs, outputs=outputs)

age_model.compile(
    optimizer='adam',
    loss='mse'
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=5,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

Gender Estimation

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
      rotation_range=25,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

In [ ]:
batch_size = 64
target_size = (64, 64)
input_shape=(64, 64, 3)
seed=1337
adam = 0.001
fre= -20
FC = 2048
E = 1
patience = 3
verbose = 1
factor = 0.50
min_lr = 0.0001
steps_per_epoch=256
validation_steps=256
epochs=8

In [ ]:
test_datagen = ImageDataGenerator( rescale = 1.0/255)

train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/AI/.kaggle/Dataset/Train',
                                                    batch_size =batch_size ,
                                                    class_mode = 'binary',
                                                    seed=seed,
                                                    target_size = target_size )     

validation_generator =  test_datagen.flow_from_directory( '/content/drive/MyDrive/AI/.kaggle/Dataset/Validation',
                                                          batch_size  = batch_size,
                                                          class_mode  = 'binary',
                                                          seed=seed,
                                                          target_size = target_size)

In [ ]:
base_model = tf.keras.applications.VGG16(input_shape=input_shape,include_top=False,weights="imagenet")

In [ ]:
# Freezing Layers

for layer in base_model.layers[:fre]:
    layer.trainable=False

In [ ]:
# Building Model
gender_model=Sequential()
gender_model.add(base_model)
gender_model.add(layers.Dropout(.2))

gender_model.add(Conv2D(512, (3, 3),strides=(1,1), activation='relu', padding='same'))
gender_model.add(BatchNormalization())
gender_model.add(layers.Dropout(.1))
gender_model.add(Conv2D(128, (3, 3),strides=(1,1), activation='relu', padding='same'))
gender_model.add(BatchNormalization())
gender_model.add(layers.Dropout(.1))
gender_model.add(Conv2D(384, (3, 3),strides=(1,1), activation='relu', padding='same'))
gender_model.add(BatchNormalization())
gender_model.add(layers.Dropout(.1))
gender_model.add(Conv2D(384, (3, 3),strides=(1,1), activation='relu', padding='same'))
gender_model.add(BatchNormalization())
gender_model.add(layers.Dropout(.1))
gender_model.add(Conv2D(500, (3, 3),strides=(1,1), activation='relu', padding='same'))
gender_model.add(BatchNormalization())
gender_model.add(MaxPooling2D(2,strides=(2,2), padding='same'))



# Add new layers
gender_model.add(Flatten())
gender_model.add(Dense(FC , activation='relu'))
gender_model.add(layers.Dropout(.2))
gender_model.add(Dense(FC , activation='relu'))
gender_model.add(layers.Dropout(.2))
gender_model.add(Dense(FC, activation='relu'))
gender_model.add(layers.Dropout(.2))
gender_model.add(Dense(E, activation='sigmoid'))

gender_model.summary()

In [ ]:
gender_model.compile(optimizer=Adam(adam),
              loss='binary_crossentropy'
              ,metrics=['accuracy'])

In [ ]:
lrd = ReduceLROnPlateau(monitor = 'val_loss',
                        patience = patience,
                        verbose = verbose ,
                        factor = factor,
                        min_lr = min_lr)

mcp = ModelCheckpoint('model.h5')

es = EarlyStopping(verbose=verbose, patience=patience)

In [ ]:
%time
hist = gender_model.fit_generator(generator=train_generator,
                           validation_data=validation_generator,
                           steps_per_epoch=steps_per_epoch,
                           validation_steps=validation_steps,
                           epochs=epochs,
                           callbacks=[lrd, mcp, es])

After Train

In [ ]:
predicted_ages = np.squeeze(model.predict(test_images))
true_ages = test_images.labels

rmse = np.sqrt(age_model.evaluate(test_images, verbose=0))
print("     Test RMSE: {:.5f}".format(rmse))

r2 = r2_score(true_ages, predicted_ages)
print("Test R^2 Score: {:.5f}".format(r2))

In [ ]:
null_rmse = np.sqrt(np.sum((true_ages - np.mean(true_ages))**2) / len(true_ages))
print("Null/Baseline Model Test RMSE: {:.5f}".format(null_rmse))

In [ ]:
test_image = '/content/drive/MyDrive/AI/AI_Project/FaceTest/test1.jpg'
image = tf.keras.preprocessing.image.load_img(test_image, target_size=(120,120))

input_arr = np.array([tf.keras.preprocessing.image.img_to_array(image)]).astype('float32') / 255
predictions = age_model.predict(input_arr)

X = tf.keras.utils.img_to_array(image)
X = np.expand_dims(X, axis=0)

images = np.vstack([X])
classes = gender_model.predict(images, batch_size=1)

plt.figure()
plt.imshow(image)

predicted_age = predictions[0][0].astype(np.int32).astype(str)
actual_age = str(25)

plt.title("Age And Gender Prediction")

print('Predicted: ' + predicted_age + '\n' +
         'Actual: ' + actual_age + '\n')
if classes[0]<0.5:
    print("This is a male")
else:
    print( "This  is a female")

plt.show()